# core

> core functions

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
class QueryResult():
    def __init__(self, 
                 query_idx: int, 
                 db_idx: int, 
                 embedding: np.ndarray, 
                 distance: float, 
                 data: dict
                ):
        self.query_idx = query_idx
        self.db_idx = db_idx
        self.embedding = embedding
        self.distance = distance
        self.data = data
        
    def to_dict(self) -> dict:
        return {
            'query_idx' : self.query_idx,
            'db_idx' : self.db_idx,
            'embedding' : self.embedding,
            'distance' : self.distance,
            'data' : self.data
        }

In [ ]:
#| export
class QueryDataset(Dataset):
    def index_with_query_idx(self, idx: int) -> Dataset:
        return self.filter(lambda row: row['query_idx']==idx)
    
    def pack_with_query_idx(self, idx: int, pack_cols: list[str]) -> dict[str, np.ndarray]:
        subset = self.index_with_query_idx(idx)
        outputs = {}
        for col_name in pack_cols:
            outputs[col_name] = np.array(subset[col_name])
            
        return outputs
    
    @classmethod
    def from_query_results(cls, query_results: list[QueryResult]):
        data_dicts = [i.to_dict() for i in query_results]
        return cls.from_list(data_dicts)

In [ ]:
query_vecs = np.random.randn(2, 256)

vector_database = np.random.randn(32, 256)

dists = ((query_vecs[:,None] - vector_database[None])**2).sum(-1)**0.5
nearest = dists.argsort(-1)[:, -5:]

query_results = []

for query_idx in range(query_vecs.shape[0]):
    for db_idx in nearest[query_idx]:
        result = QueryResult(query_idx, db_idx, vector_database[db_idx], dists[query_idx, db_idx], {})
        query_results.append(result)
        
query_dataset = QueryDataset.from_query_results(query_results)

assert query_dataset.pack_with_query_idx(0, ['embedding', 'distance'])['embedding'].shape == (5,256)

In [ ]:
#| export
class Filter():
    def __init__(self, 
                 filter_func: Callable,
                 filter_kwargs_dict: Optional[dict]=None
                ):
        self.filter_func = filter_func
        self.filter_kwargs_dict = filter_kwargs_dict if filter_kwargs_dict else {}
        
    def __call__(self, query_dataset:QueryDataset) -> QueryDataset:
        return query_dataset.filter(lambda item: self.filter_func(item), **self.filter_kwargs_dict)

In [ ]:
query_vecs = np.random.randn(2, 256)

vector_database = np.random.randn(64, 256)

dists = ((query_vecs[:,None] - vector_database[None])**2).sum(-1)**0.5
nearest = dists.argsort(-1)[:, -24:]

query_results = []

for query_idx in range(query_vecs.shape[0]):
    for db_idx in nearest[query_idx]:
        result = {
            'query_idx' : query_idx,
            'db_idx' : db_idx,
            'embedding' : vector_database[db_idx],
            'distance' : dists[query_idx, db_idx],
            'data' : {'randint': np.random.randint(0,100)}
        }

        query_results.append(result)
        
query_dataset = QueryDataset.from_list(query_results)

def simple_filter(row):
    return row['data']['randint'] < 20

f = Filter(simple_filter)
filtered_dataset = f(query_dataset)
assert len(filtered_dataset) < len(query_dataset)

f = Filter(simple_filter, {'num_proc':2})
filtered_dataset = f(query_dataset)
assert len(filtered_dataset) < len(query_dataset)

def batched_filter(batch):
    randints = np.array([i['randint'] for i in batch['data']])
    return randints < 20

f = Filter(batched_filter, {'batched':True})
filtered_dataset = f(query_dataset)
assert len(filtered_dataset) < len(query_dataset)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()